# Python for Finance 2020

MSc in Finance, Universidade Católica Portuguesa

Instructor: João Brogueira de Sousa [jbsousa@ucp.pt]

## Working with data

In this notebook, you will learn the basics of handling data with Python.

In Python, the two main resources for numerical programming are two packages: [Numpy](https://numpy.org/) and [Pandas](https://pandas.pydata.org/). 

- [Numpy](https://numpy.org/) provides tools to work with N-dimensional array objects, `numpy.ndarray`.

- [Pandas)(https://pandas.pydata.org/)

We will use [Pandas](https://pandas.pydata.org/), provides data analysis tools to efficiently handle data in tabular form.

In this notebook you will see an introduction to Pandas. You are encouraged to explore [Numpy's Quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html).

We can access both packages using `import` statements:

In [ ]:
import numpy as np # import numpy and give it a shorter name, `np`
import pandas as pd # import pandas and give it a shorter name, `pd`

### Pandas

After the `import pandas` statement above, we can access any function available in Pandas:

In [ ]:
# data from 11-02-2019 to 11-02-2020 available at https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC 

SP500 = pd.read_csv('^GSPC.csv') # you need to specify the correct path to the file being read

In [ ]:
# pd.read_csv?

In [ ]:
type(SP500)

The two main data structures in Pandas are Series (1-dimensional) and DataFrame (2-dimensional). We have just created our first DataFrame. It looks like an Excel spreadsheet:

In [ ]:
print(SP500) # this will show you a lot of data

The first thing we may want to do is to inspect how the data is organised, by looking at a small part of the DataFrame.

In [ ]:
SP500.head() # displays the top of the table

In [ ]:
SP500.tail(3) # displays the bottom 10 rows

A Dataframe can have columns with different data types:

In [ ]:
SP500.dtypes

DataFrames will distinguish a few data types: 

- Booleans (`bool`)
- Integers (`int64`)
- Floats (`float64`)
- Dates (`datetime`)
- Categorical data (`categorical`)
- Everything else (`object`)

By using Pandas we open the door to a rich collection of tools to work with data. As a quick preview:

In [ ]:
SP500.describe()

When can get different elements of a DataFrame in a variety of ways:

In [ ]:
SP500['Close'] # select a single column

In [ ]:
SP500[0:3] # select first three rows

In [ ]:
SP500.loc[0:3,['Open', 'Close']] # select columns by label

In [ ]:
SP500.iloc[0:5, 0:2] # select by position

In [ ]:
SP500[SP500['Close'] > SP500['Close'].max()*0.99] # Boolean indexing, note the .max() method to find the max Close price

Find more on DataFrame indexing in the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#indexing-selection).

We can also change an instance of a DataFrame in several ways.

In [ ]:
SP500['Open - Close'] = SP500['Open'] - SP500['Close'] # create a new column by diff of two existing columns 

In [ ]:
SP500.head()

In [ ]:
SP500['Volume'] = SP500['Volume']/1e3 # changes the values in a given column

In [ ]:
SP500.tail()

In [ ]:
SP500.drop('Open - Close', axis='columns', inplace=True) 

In [ ]:
SP500.head()

We can make these changes permanent in the DataFrame either by using the option `inplace=True` (when it's available), or assigning the output back to the variable (example above: `SP500['Volume'] = SP500['Volume']/1e3`).

#### Setting the index

We can organize the data along the row dimension also by defining an index:

In [ ]:
SP500_daily = SP500.set_index(['Date'])

In [ ]:
SP500_daily.index

In [ ]:
SP500_daily.head()

The column that was initially labeled 'Date' acquires a special role and becames the index of the DataFrame. 

We can actually have more than one index (useful if we have for example both date and company ID), and we can revert an index back to a column using the method `reset_index`.

In [ ]:
SP500_daily.reset_index(['Date'], inplace=True)
SP500_daily.head()

In [ ]:
SP500_daily = SP500.set_index(['Date']) # set again an index

SP500_daily.loc[:,'Volume'] = SP500_daily.loc[:,'Volume']/1e3 # change units on 'Volume' column

In [ ]:
SP500_daily.head()

After we have defined the index and the columns, it's straightforward to plot it and have a visual impression of the data:

In [ ]:
# Don't worry about this statement now:
%matplotlib inline

SP500_daily.plot(figsize=(10, 8)) # using the plot method on the DataFrame

We'll soon learn more about plotting with Python, but before that we'll learn about *Aggregations* and *Transforms*.

#### Built-in Aggregations

An aggregation in this context is an operation that combines multiple data values into a single value. Common examples:

- `mean`
- `var` (variance)
- `std` (st. deviation)
- `min`
- `max`
- `median`
- etc.

By using `Tab` in the usual way we can search for commonly used aggregators.

In [ ]:
SP500_daily.mean()

In [ ]:
SP500_daily.var()

In [ ]:
# SP500_daily.var?

In [ ]:
SP500_daily.var(axis=1) # does not make sense with this data set, but we can also do aggregation along the column dimension

You can also write your own aggregator and use it via the DataFrame method `agg`. See the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html#pandas.DataFrame.agg) for more details.

#### Built-in Transforms

Other usefull operations using data series may generate another series, for example calculating the percentage change between two consecutive data points. Some examples are:

- `pct_change` (percentage change)
- `diff` (difference)
- `abs` (absolute value)
- `cum|sum|prod|min|max` (commulative sum/product/min/max)
- etc.

In [ ]:
SP500_daily.pct_change().head()

In [ ]:
SP500_daily.diff().head()

Again, you can write your own transforms and use it via the DataFrame method `apply`. See the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html#pandas.DataFrame.apply) for more details.

Hence, calculating and plotting the cummulative change of a variable in pandas is straightforward:

In [ ]:
SP500_daily['Close'].pct_change().cumsum().plot(figsize=(10, 8))

### Example: Technical analysis of a trading strategy

You are being recruited by an investment firm and are asked to analyse the following investment strategy using two *simple moving averages* (SMAs). 

The strategy consists of going long on the SP500 when the shorter-term SMA is above the longer-term SMA, and going short when the opposite is true. 

The short-term SMA (`SMA1`) is calculated over a 20 day window, and the long-term (`SMA2`) over a 60 day window moving average.

With what we have learned so far, let us calculate `SMA1` and `SMA2`, illustrate it with a figure and compute the implied investment positions (long/short).

This example illustrates how fast it is to analyse this investment strategy strategy using pandas.

In [ ]:
data = pd.DataFrame(SP500_daily['Close']) # create a new DataFrame using the closing level of the SP500

type(data)

In [ ]:
data['SMA1'] = data['Close'].rolling(window=20, center=True).mean() # calculate 20 day SMA

data['SMA2'] = data['Close'].rolling(window=60, center=True).mean() # calculate 60 day SMA

In [ ]:
data.head(15) # notice the missing value

In [ ]:
data.plot(figsize=(12,8)) # notice again the missing values

In [ ]:
data.dropna(inplace=True) # drop missing data (NAN)

data.head()

In [ ]:
data['positions'] = np.where(data['SMA1'] > data['SMA2'], 1, -1) # create simple portfolio weights: 1 or -1

Finally, plot what the strategy implies in terms of positions on the underlying SP500 index:

In [ ]:
data.plot(figsize=(10,6), secondary_y='positions') # use secondary axis for portfolio weights

If you want to save your data to a `csv` file, there is a DataFrame method for that:

In [ ]:
data.to_csv('SMAs.csv') # save data to `SMAs.csv`